## Tabular Q-learning

In [1]:
import pandas as pd
import numpy as np
import time

N_STATES = 6 # state nums on one dimension
ACTIONS = ['left', 'right'] # available actions
EPSILON = .9 # epsilon greedy
ALPHA = .1 # learning rate
GAMMA = .9 # discounting factor
MAX_EPISODES = 13 # max episode number
FRESH_TIME = .3 # moving interval time

In [2]:
# Q table
# dim ->[# of states, # of actions]

def build_q_table(n_states, actions):
    table = pd.DataFrame(np.zeros((n_states, len(actions))), # initialize to all 0s
                         columns=actions # column name
                        )
    return table

build_q_table(N_STATES, ACTIONS)

,left,right
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


In [3]:
# Epsilon greedy
# epsilon is used to control the greedy degree
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :] # select the row in Q table: all actions in the current state
    if np.random.uniform()>EPSILON or state_actions.all()>0: # not greedy or the state has not been explored
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax() # greedy mode
    return action_name

In [4]:
# environment feedback
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == "right": # move right
        if S == N_STATES-2: # terminate
            S_ = "terminal"
            R = 1
        else:
            S_ = S+1
            R = 0
    else: # move left
        R = 0
        if S==0:
            S_ = S # reach the wall
        else:
            S_ = S-1
    return S_, R


In [5]:
# env update
def update_env(S, episode, step_counter):
    env_list = ['-']*(N_STATES-1) + ['T']
    if S=="terminal":
        interaction = "Episode %s: total_steps=%s" % (episode+1, step_counter)
        print("\r{}".format(interaction), end="")
        time.sleep(2)
        print("\r\t\t", end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [6]:
# RL
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table) # choose actions
            S_, R = get_env_feedback(S, A) # take actions and receive feedbacks
            q_predict = q_table.loc[S, A] # estimated (state-action) value
            if S_!="terminal":
                q_target = R + GAMMA*q_table.loc[S_,:].max() # real (state-action) value (current episode doesnot end)
            else:
                q_target = R # read (state-action) value (current episode ends)
                is_terminated = True
            
            q_table.loc[S,A] += ALPHA* (q_target - q_predict) # update q-table
            S = S_
            step_counter += 1
            update_env(S, episode, step_counter)
    return q_table

In [ ]:
# main
q_table = rl()
print("\r\nQ-table:\n")
print(q_table)

o----T